In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
%matplotlib inline

In [2]:
my_cols=['c1','c2','c3','c4','c5','c6','c7','c8','c9']
with open('../data/sample-laser-radar-measurement-data-1.txt') as f:
    table1 = pd.read_table(f, sep='\t', header=None, names=my_cols, lineterminator='\n')
    
with open('../data/sample-laser-radar-measurement-data-2.txt') as f:
    table2 = pd.read_table(f, sep='\t', header=None, names=my_cols, lineterminator='\n')
    
table = pd.concat([table1,table2])
laser_df = table[table['c1']=='L']
del laser_df['c9']
del laser_df['c1']
laser_df.columns = ['x','y','timestamp','gpx','gpy','gvx','gvy']

radar_df = table[table['c1']=='R']
del radar_df['c1']
radar_df.columns = ['rho','phi','rho_dot','timestamp','gpx','gpy','gvx','gvy']

In [3]:
laser_df.head(5)

,x,y,timestamp,gpx,gpy,gvx,gvy
1,8.44818,0.251553,1.477010e+15,8.45,0.25,-3.00027,0.00000
3,8.45582,0.253997,1.477010e+15,8.45,0.25,0.00000,0.00000
5,8.23962,0.249160,1.477010e+15,8.25,0.25,-1.81978,0.00000
7,7.84073,0.159858,1.477010e+15,7.85,0.15,-3.99972,-0.99993
9,7.54016,0.159641,1.477010e+15,7.55,0.15,-2.99982,0.00000


In [4]:
radar_df.head(5)

,rho,phi,rho_dot,timestamp,gpx,gpy,gvx,gvy
0,8.46642,0.028760,-3.040350,1.477010e+15,8.60,0.25,-3.00029,0.00000
2,8.57101,0.028232,-0.010526,1.477010e+15,8.45,0.25,0.00000,0.00000
4,8.42927,0.030143,-1.858130,1.477010e+15,8.35,0.25,-1.81979,0.00000
6,7.93510,0.023744,-3.810770,1.477010e+15,8.05,0.20,-3.99976,-0.99994
8,7.61428,0.020465,-3.220520,1.477010e+15,7.70,0.15,-2.99982,0.00000


In [5]:
xdata=np.array(laser_df['x'].values)
gpxdata=np.array(laser_df['gpx'].values)
vxdata=gpxdata-xdata
ydata=np.array(laser_df['y'].values)
gpydata=np.array(laser_df['gpy'].values)
vydata=gpydata-ydata

In [6]:
def calc_hx(gpx,gpy,gvx,gvy):
    if gpx == 0:
        return 0,0,0
    rho = math.sqrt(gpx*gpx + gpy*gpy)
    phi = math.atan(gpy/gpx)
    rho_dot = (gpx*gvx + gpy*gvy)/rho
    return np.array([rho,phi,rho_dot])

In [9]:
ghs=[ np.hstack([z[0],z[1],z[2],calc_hx(z[3],z[4],z[5],z[6])]) for z in radar_df.as_matrix(['rho','phi','rho_dot','gpx', 'gpy', 'gvx', 'gvy'])]

gr_df = pd.DataFrame(ghs)
gr_df.columns = ['rho','phi','rho_dot','grho','gphi','grho_dot']

In [10]:
gr_df.head(5)

,rho,phi,rho_dot,grho,gphi,grho_dot
0,8.46642,0.028760,-3.040350,8.603633,0.029062,-2.999023
1,8.57101,0.028232,-0.010526,8.453697,0.029577,0.000000
2,8.42927,0.030143,-1.858130,8.353742,0.029931,-1.818975
3,7.93510,0.023744,-3.810770,8.052484,0.024840,-4.023362
4,7.61428,0.020465,-3.220520,7.701461,0.019478,-2.999251
